In [ ]:
import numpy as np
import pandas as pd
from PIL import Image

In [ ]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from PIL import UnidentifiedImageError
#from keras.preprocessing.image import ImageDataGenerator
img_width, img_height = 64, 64

# Initialising the CNN

classifier = Sequential([
    Conv2D(32, (3, 3), activation='elu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='elu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='elu'),  # Adding another Convolutional layer
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='elu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='elu'),  # Increasing the number of neurons in Dense layer
    Dropout(0.25),
    Dense(256, activation='elu'),   # Adding another Dense layer
    Dense(512, activation='elu'),
    Dense(1, activation='sigmoid')
])

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Part 2 - Fitting the CNN to the images
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,      # Random rotation within the range [-20, 20] degrees
    width_shift_range=0.1,  # Random horizontal shift by up to 10% of the width
    height_shift_range=0.1, # Random vertical shift by up to 10% of the height
    shear_range=0.2,        # Shear intensity (shear angle in counter-clockwise direction in radians)
    zoom_range=0.2,         # Random zoom up to 20%
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest'     # Strategy used for filling in newly created pixels
)
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(
    '/kaggle/input/cat-and-dog/training_set/training_set',
    target_size = (64, 64),
    batch_size = 16,
    class_mode = 'binary',
)


test_set = test_datagen.flow_from_directory(
    '/kaggle/input/cat-and-dog/test_set/test_set',
    target_size = (64, 64),
    batch_size = 16,
    class_mode = 'binary',
    shuffle=False
)


classifier.fit(training_set,
epochs = 10,
validation_data = test_set,
)

In [ ]:
#Making new predictions

from keras.preprocessing import image
test_image = image.load_img('/kaggle/input/cat-again/435340919_443052074948752_5709307220134986954_n.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
    
print(prediction)

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

# Predicting classes for test data
y_pred = classifier.predict(test_set)
y_pred = (y_pred > 0.5)  # Conversion of probability (True/False)

# Real classes for data
y_true = test_set.classes

# Showing confusion matrix
cm = confusion_matrix(y_true, y_pred)
print(cm)
print(y_pred)
print(y_true)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Vizualization of confusion matrix with a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

y_pred_proba = classifier.predict(test_set)[:, 0]  

# calculate false positive rate and true positive rate at different thresholds
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_true, y_pred_proba, pos_label=1)

# calculate AUC
roc_auc = auc(false_positive_rate, true_positive_rate)

plt.title('Receiver Operating Characteristic')
# plot the false positive rate on the x axis and the true positive rate on the y axis
roc_plot = plt.plot(false_positive_rate,
                    true_positive_rate,
                    label='AUC = {:0.2f}'.format(roc_auc))

plt.legend(loc=0)
plt.plot([0,1], [0,1], ls='--')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate');